In [36]:
# retrival 
from langchain.chat_models import ChatOpenAI 
from langchain.document_loaders import UnstructuredFileLoader # 다양한 파일들 가능, 파일 종류 신경 안써도됨
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.globals import set_llm_cache, set_debug
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

set_debug(True)

llm = ChatOpenAI(
temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

# 코드 실행할대마다 임베디드 노노  
# 저장하고 불러오는 방식으로 진행 -> 한번 하고 나면 임베딩은 안바뀌니깐!
# 재귀 캐릭터 텍스트 스플리터
# tiktoken
spliter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100, # 앞 조각을 가져옴
)

loader = UnstructuredFileLoader("./files/keyword.txt")

docs = loader.load_and_split(text_splitter=spliter)

# 임베딩 모델 
embeddings = OpenAIEmbeddings()

# 캐싱하기 
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# 캐시 임베딩하기 
vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriver =vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            
            You are a helpful keyword extract assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up.
            소분류 안에 해당 요소가 있다면 해당 요소의 대분류 단어만 예를 들어 대분류가 식당이면 식당이라고만 말해.
            예시는 아래와 같아
            
            q: 맛집을 알려줘
            a: 식당
              
            q: 게스트하우스를 알려줘
            a: 숙소
              
            q: 명소를 알려줘
            a: 관광명소
            
            만약, 대분류로 식당, 숙소, 관광명소 추출할 수 없다면 이상한걸 답하지말고, TRIPTOK이라고 말해줘\n\n{context}
            
            """,
        ),
        ("human", "{question}"),
    ]
)

# 체이닝 
chain = {"context":retriver, "question":RunnablePassthrough()} | prompt | llm 

# 키워드 추출 RAG 
keyword_result = chain.invoke("근처 공원 추천해줘").content
keyword_result



[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "근처 공원 추천해줘"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "근처 공원 추천해줘"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "근처 공원 추천해줘"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "근처 공원 추천해줘"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": 

'관광명소'

In [39]:

cache_dir = LocalFileStore("./.cacheKeyword/")

# 코드 실행할대마다 임베디드 노노  
# 저장하고 불러오는 방식으로 진행 -> 한번 하고 나면 임베딩은 안바뀌니깐!
# 재귀 캐릭터 텍스트 스플리터
# tiktoken
spliter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100, # 앞 조각을 가져옴
)

loader = UnstructuredFileLoader(f"./files/{keyword_result}.txt")

docs = loader.load_and_split(text_splitter=spliter)

# 임베딩 모델 
embeddings = OpenAIEmbeddings()

# 캐싱하기 
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# 캐시 임베딩하기 
vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriver =vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful keyword extract assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up.
            내가 질문한거랑 제일 유사한  
            title : 
            keyword :
            X :
            Y :
            road_address :
            place_url :
            형식으로 말해줘 
            \n\n{context}
            
            """,
        ),
        ("human", "{question}"),
    ]
)

# 체이닝 
chain = {"context":retriver, "question":RunnablePassthrough()} | prompt | llm 


# 키워드 추출 RAG 
result = chain.invoke("공원 가고싶어").content
result

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "공원가고싶어"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "공원가고싶어"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "공원가고싶어"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "공원가고싶어"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages"

'저는 관광명소에 대한 정보만 알고 있습니다. 공원에 관한 정보는 가지고 있지 않아요. 죄송합니다.'

In [ ]:
# 키워드 추출 RAG 
keyword_result = chain.invoke("근처 박물관 추천해줘").content
keyword_result